## Query 4

> Para los productos que contienen en su descripción la palabra “stuff” (sin importar mayúsculas o minúsculas), calcular el peso total de su inventario agrupado por marca, mostrar sólo la marca y el peso total de las 5 más pesadas.

Para calcular el peso del inventario vamos a realizar los siguientes pasos:

1. Importamos el dataset de productos (data/raw/products.csv).
2. Filtramos aquellos que tienen campos invalidos (brand_id, product_id o weight in kgs) y aquellos que no contienen la palabra "stuff" en su descripción.
3. Mapeamos los campos que nos interesan (brand_id, product_id, weight_kgs) para dejar solo los valores necesarios.
4. Importamos el dataset logs de inventario para calcular el flujo de los productos (data/raw/inventory_logs.csv).
5. Filtramos aquellas entradas que tengan campos relevantes invalidos (product_id, quantity).
6. Mapeamos los campos para dejar solo los valores necesarios (product_id, quantity). Debemos acomodar quantity para que sea positivo o negativo dependiendo del tipo de movimiento.
7. Agrupamos por producto para calcular el flujo de los mismos.
8. Filtramos los que tengan un flujo valido (mayor a 0).
9. Joineamos los datasets por "product_id" para quedarnos solo con los productos que tienen la palabra "stuff" en su descripción y un flujo valido.
10. Agrupamos por marca para calcular el peso total del inventario por marca.
11. Nos quedamos solo con las 5 marcas más pesadas.

In [6]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("PesoStuffPorMarca").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

In [7]:
import re

products_df = spark.read.csv(
    "../../data/raw/products.csv", header=True, inferSchema=True
)
products_rdd = products_df.rdd

products_min = (
    products_rdd.filter(
        lambda r: getattr(r, "product_id", None) is not None
        and getattr(r, "brand", None) is not None
        and getattr(r, "weight_kg", None) is not None
        and getattr(r, "description", None) is not None
    )
    .map(
        lambda r: {
            "product_id": str(getattr(r, "product_id", "")).strip(),
            "brand": str(getattr(r, "brand", "")).strip().title(),
            "weight_kg": float(getattr(r, "weight_kg", 0.0)),
            "description": str(getattr(r, "description", "")).strip(),
        }
    )
    .filter(lambda x: x["weight_kg"] > 0.0)
)

pattern_stuff = re.compile(r"\bstuff\b", re.IGNORECASE)

stuff_products = products_min.filter(
    lambda x: pattern_stuff.search(x["description"]) is not None
).map(
    lambda x: {
        "product_id": x["product_id"],
        "brand": x["brand"],
        "weight_kg": x["weight_kg"],
    }
)

In [8]:
def normalize_from_set(val, allowed_set):
    if val is None:
        return None
    raw = str(val).strip().upper()
    return raw if raw in allowed_set else None


MOVEMENT_SIGN = {
    "IN": 1,
    "OUT": -1,
    "ADJUSTMENT": 1,
}

inv_df = spark.read.csv(
    "../../data/raw/inventory_logs.csv", header=True, inferSchema=True
)
inv_rdd = inv_df.rdd

inv_cleaned = (
    inv_rdd.filter(
        lambda r: getattr(r, "product_id", None) is not None
        and getattr(r, "movement_type", None) is not None
        and getattr(r, "quantity_change", None) is not None
    )
    .map(
        lambda r: {
            "product_id": str(getattr(r, "product_id", "")).strip(),
            "movement_type": normalize_from_set(
                getattr(r, "movement_type", None), MOVEMENT_SIGN.keys()
            ),
            "quantity_change": int(getattr(r, "quantity_change", 0) or 0),
        }
    )
    .filter(lambda x: x["movement_type"] is not None)
    .map(
        lambda x: {
            "product_id": x["product_id"],
            "qty": x["quantity_change"] * MOVEMENT_SIGN.get(x["movement_type"], 0),
        }
    )
)

flow_by_product = inv_cleaned.map(lambda x: (x["product_id"], x["qty"])).reduceByKey(
    lambda a, b: a + b
)

flow_positive = flow_by_product.filter(lambda x: x[1] > 0)

In [9]:
stuff_kv = stuff_products.map(
    lambda x: (x["product_id"], {"brand": x["brand"], "weight_kg": x["weight_kg"]})
)

joined = flow_positive.join(stuff_kv)

by_brand = joined.map(
    lambda kv: (kv[1][1]["brand"], kv[1][0] * kv[1][1]["weight_kg"])  # qty * weight
)

sum_by_brand = by_brand.reduceByKey(lambda a, b: a + b)

top5 = sum_by_brand.takeOrdered(5, key=lambda x: -x[1])

In [10]:
import pandas as pd

df_top5 = pd.DataFrame(top5, columns=["marca", "peso_total_kg"])
if not df_top5.empty:
    df_top5["peso_total_kg"] = df_top5["peso_total_kg"].round(2)
    display(df_top5)
else:
    print("No hay resultados")

,marca,peso_total_kg
0,Undefined,385686.67
1,Gardena,269784.97
2,Stubhub,255881.28
3,Leuchtturm1917,237153.79
4,Adidas,229815.13
